### Import in the necessary libraries

In [ ]:
import cv2 as cv
import threading
from time import sleep
import math
import ipywidgets as widgets
from mission_lib_pnp import Movement
from event_name_pnp import EventName
from IPython.display import display

### Import and initialize the Arm Device

In [ ]:
import Arm_Lib
Arm = Arm_Lib.Arm_Device()

initial_position = [90, 90, 0, 90, 90, 30]
Arm.Arm_serial_servo_write6_array(initial_position, 1000)

In [ ]:
e = EventName()
movement = Movement(Arm)

### Create the widgets 

In [ ]:
button_layout = widgets.Layout(width='200px', height='60px', align_self='center')

output = widgets.Output()

# Widgets
angle_input = widgets.IntSlider(min=40, max=140, step=1, value=90, layout=button_layout ) # Get the rotation angle of the robot arm.

start_button = widgets.Button(description='Start', button_style='success', layout=button_layout)

reset_button = widgets.Button(description='Reset', button_style='primary', layout=button_layout)

exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)

imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='auto'))

img_box = widgets.VBox([imgbox], layout=widgets.Layout(align_self='auto'))

Slider_box = widgets.VBox([start_button, reset_button, exit_button, angle_input], layout=widgets.Layout(align_self='auto'))

controls_box = widgets.HBox([img_box, Slider_box], layout=widgets.Layout(align_self='auto'))

### Create the event handlers for the widgets

In [ ]:
start_button.on_click(e.start_button_Callback)
reset_button.on_click(e.reset_button_Callback)
exit_button.on_click(e.exit_button_Callback)

### Create the main function camera()

In [ ]:
def camera():
    
    # Open camera
    capture = cv.VideoCapture(1)
    # Be executed in loop when the camera is opened normally 
    while True:
        try:
            _, img = capture.read()

            img = cv.resize(img, (640, 480))

            if e.action == 'Start':
                movement.move_to_positions(angle_input.value, 1000)
                e.reset()
        
            if e.action == 'Reset':
                movement.move_to_initial_position(1000)
                e.reset()
                
            if e.action == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
            sleep(0.25)
        except KeyboardInterrupt:capture.release()

In [ ]:
display(controls_box,output)
threading.Thread(target=camera, ).start()

In [ ]:
del Arm